# Base Notebook - Identity Authentication

This notebook provide basic capabilities to interface with azure open ai. It uses `identity` authentication to access the service.

It uses the Azure OpenAI SDK 2.1.0.

In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.Identity, 1.8.0"
#r "nuget: DotNetEnv, 2.5.0"


using Azure.Identity;
using Azure;

using DotNetEnv;

using System.IO;
using System.Text.Json;
using System.ClientModel;

using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;

using OpenAI.Chat;

using OpenAI.Embeddings;

In [ ]:

string _configurationFile = @"../../../configuration/.env";
Env.Load(_configurationFile);

string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") 
    ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME") 
    ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "EMBEDDING_DEPLOYMENTNAME not found";
var credential = new DefaultAzureCredential();

// Now create the client using your identity:
AzureOpenAIClient openAIClient = new AzureOpenAIClient(
    new Uri(oAiEndpoint),
    credential
);

Console.WriteLine($"OpenAI Client created with user identity at: {oAiEndpoint}, using deployment: {chatCompletionDeploymentName}");

In [ ]:
async Task<float[]> GetEmbeddingAsync(AzureOpenAIClient _openAIClient,string textToBeVecorized)
{
    // Prepare the embeddings options with the user story\n",
    EmbeddingClient embeddingClient = _openAIClient.GetEmbeddingClient(embeddingDeploymentName);
    ClientResult<OpenAIEmbedding> embeddingResult = await embeddingClient.GenerateEmbeddingAsync(textToBeVecorized);   
    float[] response = embeddingResult?.Value?.ToFloats().ToArray() ?? new float[0];
    return response;
}

In [ ]:
    float [] xx = await GetEmbeddingAsync(openAIClient,"sample text to embed");
    Console.WriteLine($"Embedding: {xx.Length}");

In [ ]:
async Task<string> CallOpenAI(AzureOpenAIClient _openAIClient, string prompt, string systemMessage, bool jsonResponse = true)
{
    // Get the chat client (using your deployment or model name)
    ChatClient chatClient = _openAIClient.GetChatClient(chatCompletionDeploymentName);

    ChatCompletionOptions chatComletionOptions = new ChatCompletionOptions(){
        MaxOutputTokenCount = 450,
        Temperature = 0.7f,
        TopP = 1.0f,
        FrequencyPenalty = 0.7f,
        PresencePenalty = 0.7f,

    };

    chatComletionOptions.ResponseFormat = jsonResponse ? ChatResponseFormat.CreateJsonObjectFormat() : ChatResponseFormat.CreateTextFormat();

    // Prepare your messages
    ChatMessage[] messages = new ChatMessage[]
    {
        new SystemChatMessage(systemMessage),
        new UserChatMessage(prompt)
    };

    // Call the chat completions endpoint with parameters directly
    ChatCompletion completions = await chatClient.CompleteChatAsync(        
    messages: messages, 
    options: chatComletionOptions);

    // Get the text from the first completion choice
    // var resp = completions.Content[0];
    
    string result = completions.Content[0].Text;
    return result;
}

In [ ]:
string systemMessage = 
@"You are an AI assistant that only answers wrong answers 
but stay in the domain of the question, 
and keep your answers short. 
For example, if the user asks for true or false, 
answer with a true or false statement. try to be funny or absurd. 
Always respond in the domain of the question.";

// Call your OpenAI method, presumably an async method that returns a string response
var resp = await CallOpenAI(openAIClient, 
    "who manufacures iPhone",
    systemMessage, false);

// Print the result
Console.WriteLine(resp);

# Playing with transcript 

Few attempts to extract insights from the transcript.

In [ ]:
string extractQuestions = 
@"You are an advanced text-extraction assistant specialized in analyzing discussion transcripts. Your goal is to identify and extract all challenges mentioned in the conversation. A “challenge” can be any problem, difficulty, obstacle, or concern that participants raise.

Instructions:
	1.	Read the entire transcript carefully. note that conversation text is within specific tags.(<v John Doe>I see you again.</v>)
	2.	Identify each statement or passage that describes a challenge or difficulty encountered or anticipated by the participants.
	3.	Summarize these challenges in a concise, clear format (bullet points or numbered list).
	4.	For each challenge:
	•	Provide a short description of what the challenge is.
	•	(Optional) If the transcript contains specific details (e.g., who raised the challenge, a brief quote, or context), include a concise snippet to illustrate it.
	5.	If no challenges are mentioned, output “No challenges found.”

Important:
	•	Only include actual challenges stated or implied by the discussion.
	•	Do not reveal any internal reasoning steps or chain-of-thought.
	•	Limit your output to the final results—the list of challenges.

Output:
    •	A list of challenges extracted from the conversation.
     The JSON must have this structure:
{
  ""Challenge"": ""<A concise phrase describing the challenge>"",
  ""Summary"": ""<A detailed explanation summarizing the challenge>""
}
    ";

In [ ]:
#!csharp
using System;
using System.IO;
using System.Text.RegularExpressions;

// Define the input file path (modify accordingly)
string filePath = "Product Leaders Co-Pilot Pt 2.vtt"; 

// Define the output file with "_cl.vtt" suffix
string outputFilePath = Path.Combine(
    Path.GetDirectoryName(filePath) ?? "",
    Path.GetFileNameWithoutExtension(filePath) + "_cl" + Path.GetExtension(filePath)
);

Console.WriteLine($"Processing: {filePath}");
Console.WriteLine($"Saving cleaned file to: {outputFilePath}");

// Read all lines from the file
var lines = File.ReadAllLines(filePath);
var cleanedLines = new List<string>();

// Regex pattern to match speaker lines with `<v SpeakerName>Text</v>`
Regex speechPattern = new Regex(@"<v\s([^>]+)>(.*?)<\/v>", RegexOptions.Compiled);

foreach (var line in lines)
{
    var match = speechPattern.Match(line);
    if (match.Success)
    {
        string speaker = match.Groups[1].Value.Trim();
        string text = match.Groups[2].Value.Trim();
        cleanedLines.Add($"{speaker}: {text}");
    }
}

// Write cleaned content to the new file
File.WriteAllLines(outputFilePath, cleanedLines);

Console.WriteLine("Cleaning completed successfully!");

In [ ]:
string filePath = "transcript1_cl.vtt";
string content = File.ReadAllText(filePath);
var resp2 = await CallOpenAI(openAIClient, 
    content,
    extractQuestions, true);

In [ ]:
Console.WriteLine(resp2);